In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
import numpy as np
import sklearn
from sklearn.cluster import KMeans, MeanShift, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
'''
Purpose: Count missing values and columns
Parameters: 
  df_in: Dataframe input need to detect missing value
Returns:
  None
#'''

In [ ]:
def detect_missing_values(df_in):
  df_miss = df_in.isnull().sum()
  #print(df_miss)
  df_miss = df_miss[df_miss>0]
  print("There are {0} columns have missing values:\n{1}".format(len(df_miss),df_miss))

In [ ]:
'''
Purpose: Drop missing value
Parameters: 
  df_in: Dataframe input need to drop
  list_columns: List of columns need to detect and drop missing value
Returns:
  Dataframe after droping missing value
#'''

In [ ]:
def drop_missing_values(df_in,list_columns):
  df_out = df_in.copy()
  #print(df_out)
  #print(list_columns)
  count_missing = 0
  for column in list_columns:
    try:
      count_missing = count_missing + df_out[column].isnull().sum()
      #print("Remove {0} missing values from {1} column".format(df_out[column].isnull().sum(),column))
      df_out = df_out[~df_out[column].isnull()].copy()
    except:
      print("Some thing error with column {0}".format(column))
  print("There are {0} values have been removed".format(count_missing))
  return df_out

In [ ]:
'''
Purpose: Replace null values with most frequency value
Parameters: 
  df_in: Dataframe input
  list_columns: List of columns need to detect missing value and replace
Returns:
  Dataframe after replace null values with most frequency value
#'''

In [ ]:
def impute_mode_value(df_in, list_columns):
  df_out = df_in.copy()
  count_missing = 0
  for column in list_columns:
    try:
      count_missing = count_missing + df_out[column].isnull().sum()
      #print("Impute {0} missing values from {1} column".format(df_out[column].isnull().sum(),column))
      mode_value = df_out[column].mode()[0]
      df_out[column].fillna(mode_value,inplace = True)
    except:
      print("Some thing error with column {0}".format(column))
  print("There are {0} values have been imputed".format(count_missing))
  return df_out

In [ ]:
'''
Purpose: Drop feature with highly correlation
Parameters: 
  df_in: Dataframe input
Returns:
  df_in: If there ain't any highly correlation features
  Dataframe: If there are highly correlation features, return dataframe after 
  remove feature
#'''

In [ ]:
def drop_highly_corr_feature(df_in):
  df_out = df_in.copy()
  corr = df_out.corr()
  upper = corr.where(np.triu(np.ones(corr.shape),k=1).astype(np.bool))
  to_drop = [column for column in upper.columns if any(abs(upper[column])>0.95)]
  print("There are {0} highly correlation feature has been removed\n{1}".format(len(to_drop),to_drop))
  if (len(to_drop)==0):
    return df_in
  else:
    return df_out.drop(to_drop,axis=1)

In [ ]:
'''
Purpose: Drop feature with high null values
Parameters: 
  df_in: Dataframe input
  threshold: threshold to remove feature
Returns:
  Remove columns if percentage of nan greater than threshold. Return dataframe 
  after remove columns
#'''

In [ ]:
def drop_high_nan_column(df_in,threshold = .8):
  res = df_in.loc[:,df_in.isnull().mean()<threshold]
  before = df_in.shape[1]
  after = res.shape[1]
  print("There are {0} columns have been removed because of high nan percentage".format(before-after))
  return res

In [ ]:
'''
Purpose: Remove outlier
Parameters: 
  df_in: Dataframe input
Returns:
  Dataframe after remove outlier
#'''

In [ ]:
def remove_outlier(df_in):
  Q1 = df_in.quantile(0.25)
  Q3 = df_in.quantile(0.75)
  IQR = Q3-Q1
  outlier_condition = (df_in<(Q1-1.5*IQR))|(df_in>(Q3+1.5*IQR))
  df_out = df_in[~outlier_condition]
  before = df_in.isnull().sum().sum()
  after = df_out.isnull().sum().sum()
  print("There are {0} outliers have been removed".format(after-before))
  return df_out

In [ ]:
'''
Purpose: Generate next permutation
Parameters: 
  a: A state
Returns:
  Next state of a
#'''

In [ ]:
def next_permutation(a):
  for i in range(len(a)-2,-1,-1):
    if (a[i]<a[i+1]):
      break
  if (a[i]>a[i+1]):
      return False
  for j in range(len(a)-1,-1,-1):
      if (a[j]>a[i]):
        break
  a[i], a[j] = a[j], a[i]
  a[i+1:] = reversed(a[i+1:])
  return True

In [ ]:
'''
Purpose: Find label to transform y_pred to maximize accuracy with y_true
Parameters: 
  y_true: Array of true label 
  y_pred: Array of predict label
Returns:
  Best label to tranform y_pred
#'''

In [ ]:
def find_best_label(y_true, y_pred):
  y_temp = y_true.copy()
  yp_temp = y_pred.copy()
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  unique = np.unique(y_true)
  #*********************************
  #print(y_true)
  #print(y_pred)
  #*********************************
  label = []
  for i in range(0,len(unique)):
    label.append(i)
  max_count = 0
  best_label = []
  while (True):
    counts = 0
    for i in range(0,len(y_true)):
      if (label[y_pred[i]] == y_true[i]):
        counts = counts + 1
    if (counts>max_count):
      max_count = counts
      best_label = label.copy()
    if (not next_permutation(label)):
      y_true = y_temp
      return best_label
  y_true = y_temp
  y_pred = yp_temp
  return best_label

In [ ]:
'''
Purpose: Transform y to a new vector with label
Parameters: 
  y: Array of label
  label: Label to transform
Returns:
  Vector after transform
#'''

In [ ]:
def change_label(y,label):
  res = []
  for i in range(0,len(y)):
    res.append(label[y[i]])
  return res

In [ ]:
'''
Purpose: Train data with Kmeans model then print accuracy
Parameters: 
  X_train: Train set feature
  y_train: Train set label
  X_test: Test set feature
  y_test: Test se label
Returns:
  None
#'''

In [ ]:
def KMeans_model(X_train,y_train,X_test,y_test):
  unique = np.unique(y_train)
  cls = KMeans(n_clusters = len(unique), max_iter = len(y_train),random_state=42)
  cls = cls.fit(X_train)
  y_pred_train = np.array(cls.labels_)
  label = find_best_label(y_train,y_pred_train)
  y_pred_train = change_label(y_pred_train,label)
  y_pred = cls.predict(X_test)
  y_pred = change_label(y_pred,label)
  print("Accuracy train: ",accuracy_score(y_train,y_pred_train))
  print("Accuracy test: ", accuracy_score(y_test,y_pred))
  print("Confusion_matrix: ", confusion_matrix(y_test,y_pred))

In [ ]:
'''
Purpose: Train data with Gaussian Mixture model then print accuracy
Parameters: 
  X_train: Train set feature
  y_train: Train set label
  X_test: Test set feature
  y_test: Test se label
Returns:
  None
#'''

In [ ]:
def GaussianMixture_model(X_train,y_train,X_test,y_test):
  unique = np.unique(y_train)
  cls = GaussianMixture(n_components= len(unique), max_iter = len(y_train),random_state=42)
  cls = cls.fit(X_train)
  y_pred_train = np.array(cls.predict(X_train))
  label = find_best_label(y_train,y_pred_train)
  y_pred_train = change_label(y_pred_train,label)
  y_pred = cls.predict(X_test)
  y_pred = change_label(y_pred,label)
  print("Accuracy train: ",accuracy_score(y_train,y_pred_train))
  print("Accuracy test: ", accuracy_score(y_test,y_pred))
  print("Confusion_matrix: ", confusion_matrix(y_test,y_pred))

In [ ]:
'''
Purpose: Train data with Agglomerative Clustering model then print accuracy
Parameters: 
  X_train: Train set feature
  y_train: Train set label
  X_test: Test set feature
  y_test: Test se label
Returns:
  None
#'''

In [ ]:
def AgglomerativeClustering_model(X_train,y_train,X_test,y_test):
  unique = np.unique(y_train)
  cls = AgglomerativeClustering(n_clusters= len(unique))
  cls = cls.fit(X_train)
  y_pred_train = np.array(cls.labels_)
  label = find_best_label(y_train,y_pred_train)
  y_pred_train = change_label(y_pred_train,label)
  y_pred = cls.fit_predict(X_test)
  y_pred = change_label(y_pred,label)
  print("Accuracy train: ",accuracy_score(y_train,y_pred_train))
  print("Accuracy test: ", accuracy_score(y_test,y_pred))
  print("Confusion_matrix: ", confusion_matrix(y_test,y_pred))

In [ ]:
def main():
  df = pd.read_csv('/content/drive/My Drive/Datasets/Py4DS_Lab1/diabetes.csv')
  #print(df)
  '''
  # Add some outlier value to datafram
  n = int(np.random.rand()*100)
  print("Create {0} outliers values".format(n))
  for k in range(0,n):
    i = int(np.random.rand()*df.shape[0])
    j = int(np.random.rand()*df.shape[1])
    df.iloc[i,j] = -10000000
  #'''
  '''
  # Add some random nan value to datafram
  n = int(np.random.rand()*100)
  print("Create {0} nan values".format(n))
  for k in range(0,n):
    i = int(np.random.rand()*df.shape[0])
    j = int(np.random.rand()*df.shape[1])
    df.iloc[i,j] = np.nan
  #'''
  '''
  # Add a nan columns to data frame
  df['nan'] = np.full([df.shape[0]],np.nan)
  #'''
  detect_missing_values(df)
  df = drop_highly_corr_feature(df)
  df = remove_outlier(df)
  df = drop_high_nan_column(df,0.8)
  #df = impute_mode_value(df,df.columns)
  df = drop_missing_values(df,df.columns)
  for col in df.columns:
    print("*"*30)
    print(df[col].describe())
  print(df.dtypes)
  X = df.drop(['Outcome'],axis=1)
  y = df['Outcome']
  X_train, X_test, y_train, y_test  = train_test_split(X,y, test_size = 0.2,random_state=42)
  print("*"*40)
  print("KMeans clustering model: ")
  KMeans_model(X_train,y_train,X_test,y_test)
  print("*"*40)
  print("Gaussian Mixture clustering model: ")
  GaussianMixture_model(X_train,y_train,X_test,y_test)
  print("*"*40)
  print("Agglomerative Clustering model: ")
  AgglomerativeClustering_model(X_train,y_train,X_test,y_test)
  #'''

In [ ]:
main()

There are 0 columns have missing values:
Series([], dtype: int64)
There are 0 highly correlation feature has been removed
[]
There are 146 outliers have been removed
There are 0 columns have been removed because of high nan percentage
There are 129 values have been removed
******************************
count    639.000000
mean       3.804382
std        3.260995
min        0.000000
25%        1.000000
50%        3.000000
75%        6.000000
max       13.000000
Name: Pregnancies, dtype: float64
******************************
count    639.000000
mean     119.112676
std       29.162175
min       44.000000
25%       99.000000
50%      114.000000
75%      137.000000
max      198.000000
Name: Glucose, dtype: float64
******************************
count    639.000000
mean      72.120501
std       11.348686
min       38.000000
25%       64.000000
50%       72.000000
75%       80.000000
max      106.000000
Name: BloodPressure, dtype: float64
******************************
count    639.000000
me